In [1]:
import pandas as pd
from scripts.feature_engineering import extract_features, compute_orders_in_circles

# Load datasets
completed_orders = pd.read_csv('data/completed_orders.csv')

# Feature Extraction
completed_orders = extract_features(completed_orders)
completed_orders.to_csv('data/completed_orders_with_features.csv', index=False)
completed_orders.describe

<bound method NDFrame.describe of         Trip ID                         Trip Origin  \
0        391996  6.508813001668548,3.37740316890347   
1        391997                 6.4316714,3.4555375   
2        391998         6.631679399999999,3.3388976   
3        391999         6.572757200000001,3.3677082   
4        392001                 6.6010417,3.2766339   
...         ...                                 ...   
535994  1637696         6.448218499999999,3.4772075   
535995  1637702         6.442320899999999,3.4736868   
535996  1637704                  6.4281982,3.492248   
535997  1637705                 6.5869296,3.3632966   
535998  1637709         6.647209999999999,3.4851489   

                            Trip Destination     Trip Start Time  \
0                6.650969799999999,3.3450307 2021-07-01 07:28:04   
1            6.4280814653326,3.4721885847586 2021-07-01 06:38:04   
2                6.508324099999999,3.3590397 2021-07-01 06:21:02   
3                6.58488109999999

In [3]:
delivery_requests = pd.read_csv('data/driver_locations_during_request.csv')
delivery_requests = compute_orders_in_circles(delivery_requests, 'lat', 'lng')
delivery_requests.to_csv('riders_and orders_in_500m_circle.csv')
delivery_requests.describe

In [ ]:
# Cluster Analysis for Delivery Locations
def cluster_analysis(df, lat_col, lng_col, n_clusters=10):
    coords = df[[lat_col, lng_col]].values
    kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(coords)
    df['cluster'] = kmeans.labels_
    return df

delivery_requests = cluster_analysis(delivery_requests, 'lat', 'lng')
delivery_requests.head

In [ ]:
# Visualization with Datashader
def plot_clusters(df, lat_col, lng_col, cluster_col):
    cvs = ds.Canvas(plot_width=800, plot_height=800)
    agg = cvs.points(df, lat_col, lng_col, ds.count_cat(cluster_col))
    img = tf.shade(agg, how='eq_hist')
    export_image(img, "clusters", background="white")
    return img

# Call the visualization function
cluster_img = plot_clusters(delivery_requests, 'lat', 'lng', 'cluster')

# Display the image
plt.imshow(cluster_img.to_pil())
plt.axis('off')
plt.show()


In [ ]:
from geopy.distance import geodesic

# Computing Routes between Points (Simplified)
# This is a placeholder. Actual routing would require integration with a routing API like OSRM or Google Maps.
def compute_route_distance(origin, destination):
    return geodesic(origin, destination).km  # Simplified; actual route might be longer

# Example of computing route distance
completed_orders['Route Distance'] = completed_orders.apply(lambda row: compute_route_distance((row['Trip Origin Lat'], row['Trip Origin Lng']), (row['Trip Destination Lat'], row['Trip Destination Lng'])), axis=1)

completed_orders['Route Distance'].head